<center>
<h1> TP 5-6 de Recherche opérationnelle </h1>
<h1> Année 2020-2021 - 2e année département Sciences du Numérique </h1>
<h1> EL BOUCHIBTI Aymane</h1>
<h1> AKBY Amine </h1>    
</center>

# Prise en main : ordonnancement avec contraintes de précédence

Récupération des données

In [1]:
D=[2, 3, 1, 4, 1]

5-element Array{Int64,1}:
 2
 3
 1
 4
 1

#### Modélisation et résolution de (P1) à l'aide d'un programme linéaire :

In [2]:
using JuMP
using Clp

model = Model(Clp.Optimizer) # set optimizer
set_optimizer_attribute(model, "LogLevel", 0) #don't display anything during solve
set_optimizer_attribute(model, "Algorithm", 4) #LP solver chosen is simplex

# define t variables
@variable(model, t[i in 1:5] >= 0)
@variable(model, tfin)

# define objective function
@objective(model, Min, tfin)

# define constraints: t_j - t_i  >= D[i], \forall i predecesseur de j
@constraint(model, t[2] - t[1] >= D[1] )
@constraint(model, t[3] - t[1] >= D[1] )
@constraint(model, t[4] - t[2] >= D[2] )
@constraint(model, t[4] - t[3] >= D[3] )
@constraint(model, t[5] - t[3] >= D[3] )

#define constraints: tfin - t_i >= Duree[i], \forall i predecesseur de j
@constraint(model, tfin - t[4] >= D[4] )
@constraint(model, tfin - t[5] >= D[5] )

println(model)

print("start solve ... ")
optimize!(model)
print("... end solve")


println("\n\nSolution PL:\n \t t=", value.(t), "\t tfin=", value(tfin))

Min tfin
Subject to
 -t[1] + t[2] >= 2.0
 -t[1] + t[3] >= 2.0
 -t[2] + t[4] >= 3.0
 -t[3] + t[4] >= 1.0
 -t[3] + t[5] >= 1.0
 -t[4] + tfin >= 4.0
 -t[5] + tfin >= 1.0
 t[1] >= 0.0
 t[2] >= 0.0
 t[3] >= 0.0
 t[4] >= 0.0
 t[5] >= 0.0

start solve ... ... end solve

Solution PL:
 	 t=[0.0, 2.0, 2.0, 5.0, 8.0]	 tfin=9.0


#### Modélisation classique par graphe potentiel-tache 

On modélise le problème précedent par un graphe où les durées sont présentées par des arcs et les tâches par des noeuds du graphe.
Résoudre le problème (P1) revient à trouver le plus long chemin dans le graphe entre le début et la fin.
La valeur du plus long chemin est la durée de fin du projet.

In [3]:
function PlusLongChemin(V, w, s)
    D = w[findall(x->x==s, V)[1],:]
    for t in 1:length(V)
        for x in 1:length(V)
            if max(D[t], D[x] + w[x, t]) != Inf
                D[t] = max(D[t], D[x] + w[x, t])
            else
                D[t] = min(D[t], D[x] + w[x, t])
            end
        end
    end
    return D
end

PlusLongChemin (generic function with 1 method)

In [4]:
# Test
w = [0 2 2 Inf Inf Inf;
     Inf 0 Inf 3 Inf Inf;
     Inf Inf 0 1 1 Inf;
     Inf Inf Inf 0 Inf 4;
     Inf Inf Inf Inf 0 1;
     Inf Inf Inf Inf Inf 0]
V = ['A','B','C','D','E','F']

f = PlusLongChemin(V, w, 'A')
println("\n\nSolution PL:\n \t t=", value.(f), "\t tfin=", value(f[6]))



Solution PL:
 	 t=[0.0, 2.0, 2.0, 5.0, 3.0, 9.0]	 tfin=9.0


La solution obtenue par l'algorithme de Bellman Ford du plus long chemin est la même pour le programme linéaire.
On obtient une durée de fin de projet : 9.

# Job-shop : ordonnancement avec contraintes de précédence et contraintes de ressources

#### Relaxation des contraintes de ressources

Soit une relaxation (R) qui consiste à ignorer les contraintes de ressources du job-shop. (R) est équivalente au problème (P1).

| Tâche i | A1 | A2 | A3 | B1 | B2 | B3 |
| --- | --- | --- | --- | --- | --- |--- |
| Durée di | 6 | 7 | 0 | 3 | 5 | 1 |
| Conditions de début | Vrai | Après fin de A1 | Après fin de A2 | Vrai | Après fin de B1 | Après fin de B2 |

In [5]:
D=[6, 7, 0, 3, 5, 1]

6-element Array{Int64,1}:
 6
 7
 0
 3
 5
 1

In [6]:
using JuMP
using Clp

model = Model(Clp.Optimizer) # set optimizer
set_optimizer_attribute(model, "LogLevel", 0) #don't display anything during solve
set_optimizer_attribute(model, "Algorithm", 4) #LP solver chosen is simplex

# define t variables
@variable(model, t[i in 1:6] >= 0)
@variable(model, tfin)

# define objective function
@objective(model, Min, tfin)

# define constraints: t_j - t_i  >= D[i], \forall i predecesseur de j
@constraint(model, t[2] - t[1] >= D[1])
@constraint(model, t[3] - t[2] >= D[2])
@constraint(model, t[5] - t[4] >= D[4])
@constraint(model, t[6] - t[5] >= D[4])

#define constraints: tfin - t_i >= Duree[i], \forall i predecesseur de j
@constraint(model, tfin - t[6] >= D[6])
@constraint(model, tfin - t[3] >= D[3])

println(model)

print("start solve ... ")
optimize!(model)
print("... end solve")


println("\n\nSolution PL:\n \t t=", value.(t), "\t tfin=", value(tfin))

Min tfin
Subject to
 -t[1] + t[2] >= 6.0
 -t[2] + t[3] >= 7.0
 -t[4] + t[5] >= 3.0
 -t[5] + t[6] >= 3.0
 -t[6] + tfin >= 1.0
 -t[3] + tfin >= 0.0
 t[1] >= 0.0
 t[2] >= 0.0
 t[3] >= 0.0
 t[4] >= 0.0
 t[5] >= 0.0
 t[6] >= 0.0

start solve ... ... end solve

Solution PL:
 	 t=[0.0, 6.0, 13.0, 0.0, 3.0, 6.0]	 tfin=13.0


In [7]:
function PlusLongChemin(V, w, s)
    D = w[findall(x->x==s, V)[1],:]
    for t in 1:length(V)
        for x in 1:length(V)
            if max(D[t], D[x] + w[x, t]) != Inf
                D[t] = max(D[t], D[x] + w[x, t])
            else
                D[t] = min(D[t], D[x] + w[x, t])
            end
        end
    end
    return D
end

PlusLongChemin (generic function with 1 method)

In [8]:
# Test
graphe = [0   6   Inf Inf Inf Inf;
          Inf 0   Inf Inf Inf 7;
          Inf Inf 0   3   Inf Inf;
          Inf Inf Inf 0   5   Inf;
          Inf Inf Inf Inf 0   1;
          Inf Inf Inf Inf Inf 0]

V = ["A1","A2","B1","B2","B3","Fin"]

f = PlusLongChemin(V, graphe, "A1")
println("\n\nSolution PL:\n \t t=", value.(f), "\t tfin=", value(f[6]))



Solution PL:
 	 t=[0.0, 6.0, Inf, Inf, Inf, 13.0]	 tfin=13.0


Les opérations 1 des travaux 1 et 2 commencent en même temps alors qu'ils utilisent la même ressource selon les contraintes. Donc, la solution donnée par le programme linéaire ne respecte pas les contraintes du tableau (b).

# Résolution avec Méthode des graphes disjonctifs

### PSE basée sur la relaxation linéaire du modèle avec bigM

Récupération des données

In [9]:
MachinesParJob=[1 3 0; 1 2 3]
Duree=[6 7 0; 3 5 1]
bigM=sum(Duree)

22

Procédure d'application des tests de sondabilités TA, TO et TR pour le cas de la relaxation linéaire

In [10]:
function TestsSondabilite_relaxlin(model2, varsbin, BestTfin, Bestsol)
    TA, TO, TR = false, false, false
    if(termination_status(model2) == MOI.INFEASIBLE)#Test de faisabilite
        TA=true
        println("TA")
    elseif(objective_value(model2) >= BestTfin) #Test d'optimalite
        TO=true
        println("TO")
    elseif( prod(abs.([round.(v, digits=0) for v in value.(varsbin)]-value.(varsbin)) .<= fill(10^-5, size(varsbin))) 
        ) #Test de resolution
        TR=true
        println("TR")
        if (value(tfin) <= BestTfin)
            Bestsol = value.(t)
            BestTfin=value(tfin)
        end
    else
        println("non sondable")
    end
    TA, TO, TR, Bestsol, BestTfin
end

TestsSondabilite_relaxlin (generic function with 1 method)

Procédure de séparation et stratégie d'exploration permettant de se placer au prochain noeud à traiter

In [11]:

function SeparerNoeud_relaxlin(varsshouldbebinary, listvars, listvals)
    # le noeud est non-sondable. Appliquer le critère de séparation pour le séparer en sous-noeuds 
    # et choisir un noeud-fils le plus à gauche   
    
    #find a fractionnal variable
    i, var = 1, 0
    while((i <= length(varsshouldbebinary)) && (var==0))
        #if(varsshouldbebinary[i] ∉ listvars)
        if(abs(round(value(varsshouldbebinary[i]), digits=0) - value(varsshouldbebinary[i]) ) >= 10^-5)
            var=varsshouldbebinary[i]
        end
        i+=1
    end
    
    #=
    #find most fractionnal variable
    i, var, maxfrac = -1, 0, 0.0
    for i in 1:length(varsshouldbebinary)
        if(abs(round(value(varsshouldbebinary[i]), digits=0) - value(varsshouldbebinary[i]) ) >= maxfrac) 
            #if a variable is more fractinonal
            var=varsshouldbebinary[i]
            maxfrac=abs(round(value(varsshouldbebinary[i]), digits=0) - value(varsshouldbebinary[i]) )
            #println(i, " ", var, " ", maxfrac)
        end
    end
    =#
    

    set_lower_bound(var,1.0)
    set_upper_bound(var,1.0)

    push!(listvars,var) #stocker l'identite de la variable choisie pour la séparation
    push!(listvals,1.0) #stocker la branche choisie, identifiee par la valeur de la variable choisie
    listvars, listvals
end


function ExplorerAutreNoeud_relaxlin(listvars, listvals)
    #this node is sondable, go back to parent node then right child if possible
    
    stop=false
    #go back to parent node
    var=pop!(listvars)
    theval=pop!(listvals)
    set_lower_bound(var,0.0)
    set_upper_bound(var,1.0)

    #go to right child if possible, otherwise go back to parent
    while( (theval==0.0) && (length(listvars)>= 1))
        var=pop!(listvars)
        theval=pop!(listvals)
        set_lower_bound(var,0.0) 
        set_upper_bound(var,1.0)
    end
    if theval==1.0
        set_lower_bound(var,0.0)
        set_upper_bound(var,0.0)
        push!(listvars,var)
        push!(listvals,0.0)
    else
        println("\nFINISHED")
        stop=true
    end
    listvars, listvals, stop 
end

ExplorerAutreNoeud_relaxlin (generic function with 1 method)

Creation de la relaxation linéaire (= modèle associé au noeud 0)

In [12]:
# ROOT NODE
using JuMP
using Clp

model2 = Model(Clp.Optimizer) # set optimizer
set_optimizer_attribute(model2, "LogLevel", 0) #don't display anything during solve
set_optimizer_attribute(model2, "Algorithm", 4) #LP solver chosen is simplex

# define t variables
@variable(model2, t[i in 1:2, j in 1:3] >= 0)
@variable(model2, tfin)

# define objective function
@objective(model2, Min, tfin)

# define constraints: t_i(j+1) - t_ij  >= Duree[i,j], \forall i,j
@constraint(model2, t[1,2] - t[1,1] >= Duree[1,1] )
@constraint(model2, t[1,3] - t[1,2] >= Duree[1,2] )
@constraint(model2, t[2,2] - t[2,1] >= Duree[2,1] )
@constraint(model2, t[2,3] - t[2,2] >= Duree[2,2] )

#define constraints: tfin - t_ij >= Duree[i,j], \forall ij
@constraint(model2, tfin - t[1,3] >= Duree[1,3] )
@constraint(model2, tfin - t[2,3] >= Duree[2,3] )


# define x variables as CONTINUOUS (recall that it is not possible to define binary variables in Clp)
@variable(model2, 0 <= x_2_1__1_1 <= 1)
@variable(model2, 0 <= x_2_3__1_2 <= 1)
varsshouldbebinary=[x_2_1__1_1,x_2_3__1_2]


# define bigM constraints linking x and t variables
@constraint(model2, t[2,1] - t[1,1] >=  Duree[1,1] - bigM*x_2_1__1_1)
@constraint(model2, t[1,1] - t[2,1] >=  Duree[2,1] - bigM*(1-x_2_1__1_1))
@constraint(model2, t[2,3] - t[1,2] >=  Duree[1,2] - bigM*x_2_3__1_2)
@constraint(model2, t[1,2] - t[2,3] >=  Duree[2,3] - bigM*(1-x_2_3__1_2))


println(model2)

Min tfin
Subject to
 -t[1,1] + t[1,2] >= 6.0
 -t[1,2] + t[1,3] >= 7.0
 -t[2,1] + t[2,2] >= 3.0
 -t[2,2] + t[2,3] >= 5.0
 -t[1,3] + tfin >= 0.0
 -t[2,3] + tfin >= 1.0
 -t[1,1] + t[2,1] + 22 x_2_1__1_1 >= 6.0
 t[1,1] - t[2,1] - 22 x_2_1__1_1 >= -19.0
 -t[1,2] + t[2,3] + 22 x_2_3__1_2 >= 7.0
 t[1,2] - t[2,3] - 22 x_2_3__1_2 >= -21.0
 t[1,1] >= 0.0
 t[2,1] >= 0.0
 t[1,2] >= 0.0
 t[2,2] >= 0.0
 t[1,3] >= 0.0
 t[2,3] >= 0.0
 x_2_1__1_1 >= 0.0
 x_2_3__1_2 >= 0.0
 x_2_1__1_1 <= 1.0
 x_2_3__1_2 <= 1.0



Boucle principale : résoudre la relaxation linéaire, appliquer les tests de sondabilité, identifier le prochain noeud, répéter.

In [13]:

listvars=[]
listvals=[]

BestTfin=bigM
Bestsol=[]

current_node_number=0
stop = false

while(!stop)
    
    println("\nNode number ", current_node_number, ": \n-----\n", model2)

    print("Solve : start ... ")
    status = optimize!(model2)
    println("... end")

    println("\nSolution relax lin",); [print("\t", name(v),"=",value(v)) for v in all_variables(model2)]; 
    println(" "); println("\nSolution precedemment memorisee ", Bestsol, " avec date de fin ", BestTfin, "\n")

    TA, TO, TR, Bestsol, BestTfin = TestsSondabilite_relaxlin(model2, varsshouldbebinary, BestTfin, Bestsol)

    is_node_sondable = TA || TO || TR
    
    if(!is_node_sondable)
        listvars, listvals = SeparerNoeud_relaxlin(varsshouldbebinary, listvars, listvals)
    else
        listvars, listvals, stop = ExplorerAutreNoeud_relaxlin(listvars, listvals)
    end

    current_node_number = current_node_number + 1
end

println("\n******\n\nOptimal value = ", BestTfin, "\n\nOptimal t=", Bestsol)



Node number 0: 
-----
Min tfin
Subject to
 -t[1,1] + t[1,2] >= 6.0
 -t[1,2] + t[1,3] >= 7.0
 -t[2,1] + t[2,2] >= 3.0
 -t[2,2] + t[2,3] >= 5.0
 -t[1,3] + tfin >= 0.0
 -t[2,3] + tfin >= 1.0
 -t[1,1] + t[2,1] + 22 x_2_1__1_1 >= 6.0
 t[1,1] - t[2,1] - 22 x_2_1__1_1 >= -19.0
 -t[1,2] + t[2,3] + 22 x_2_3__1_2 >= 7.0
 t[1,2] - t[2,3] - 22 x_2_3__1_2 >= -21.0
 t[1,1] >= 0.0
 t[2,1] >= 0.0
 t[1,2] >= 0.0
 t[2,2] >= 0.0
 t[1,3] >= 0.0
 t[2,3] >= 0.0
 x_2_1__1_1 >= 0.0
 x_2_3__1_2 >= 0.0
 x_2_1__1_1 <= 1.0
 x_2_3__1_2 <= 1.0

Solve : start ... ... end

Solution relax lin
	t[1,1]=0.0	t[2,1]=0.0	t[1,2]=6.0	t[2,2]=3.0	t[1,3]=13.0	t[2,3]=8.0	tfin=13.0	x_2_1__1_1=0.27272727272727276	x_2_3__1_2=0.22727272727272732 

Solution precedemment memorisee Any[] avec date de fin 22

non sondable

Node number 1: 
-----
Min tfin
Subject to
 -t[1,1] + t[1,2] >= 6.0
 -t[1,2] + t[1,3] >= 7.0
 -t[2,1] + t[2,2] >= 3.0
 -t[2,2] + t[2,3] >= 5.0
 -t[1,3] + tfin >= 0.0
 -t[2,3] + tfin >= 1.0
 -t[1,1] + t[2,1] + 22 x_2_1_

In [14]:
println("\n******\n\nOptimal value = ", BestTfin, "\n\nOptimal t=", Bestsol)


******

Optimal value = 15.0

Optimal t=[0.0 6.0 15.0; 6.0 9.0 14.0]


### PSE basée sur le graphe disjonctif

In [61]:
function PlusLongChemin(V, w, w_disjonctif, s)
    D = w[findall(x->x==s,V)[1],:]
    for t in 1:length(V)
        for x in 1:length(V)
            if max(D[t],D[x] + w[x, t]) !=  Inf
                D[t] = max(D[t], D[x] + w[x, t])
            else
                D[t] = min(D[t], D[x] + w[x, t])
            end
            if (w[x,t] != Inf) && (w[x,t] > 0)
                for r in 1:length(V)
                    if w_disjonctif[r,t] != Inf
                        if (D[t] + w_disjonctif[t,r] < D[r] + w_disjonctif[r,t])
                            D[t] = min(D[t] + w_disjonctif[r,t],D[r] + w_disjonctif[r,t])
                        end
                    end
                end
            end
        end
    end
    D = D+ones(length(V))
    return D
end

PlusLongChemin (generic function with 3 methods)

In [63]:
# Test
graphe = [0    6     Inf   Inf   Inf   Inf;
          Inf  0     Inf   Inf   Inf     7;
          Inf  Inf   0     3     Inf   Inf;
          Inf  Inf   Inf   0     5     Inf;
          Inf  Inf   Inf   Inf   0       1;
          Inf  Inf   Inf   Inf   Inf     0]

graphe_disjoint = [0    Inf   6     Inf   Inf   Inf;
                   Inf  0     Inf   Inf   7     Inf;
                   3    Inf   0     Inf   Inf   Inf;
                   Inf  Inf   Inf   0     Inf   Inf;
                   Inf  1     Inf   Inf   0     Inf;
                   Inf  Inf   Inf   Inf   Inf     0]

V = ["A1","A2","B1","B2","B3","Fin"]

res = PlusLongChemin(V, graphe,graphe_disjoint,"A1")

println("\n tfin = ", value(res[6]))


 tfin = 15.0
